Let’s start with something simple: the stack of two layers we used in the previous section. Its Functional API version looks like the following listing.

In [1]:
import keras
from keras import layers, models

#create an input layer (symbolic tensor)
inputs = keras.Input(shape=(3,), name="my_input")
#create a layer and call it with the input
features = layers.Dense(64, activation="relu")(inputs)

outputs = layers.Dense(10, activation="softmax")(features)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_input (InputLayer)           │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 906 (3.54 KB)

 Trainable params: 906 (3.54 KB)

 Non-trainable params: 0 (0.00 B)

the functional API is a way to create models that are more flexible than the Sequential API.

they hold more complex topologies, such as multi-input models and multi-output models.

example: 

Let’s say you’re building a system to rank customer support tickets by priority and route them to the appropriate department. Your model has three inputs:

- The title of the ticket (text)
- The text of the ticket (text)
- The priority of the ticket (categorical)

the model has two outputs:
- The priority of the ticket (categorical)
- The department to route the ticket to (categorical)

In [2]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

#define model inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

#combine input features into a single tensor
features = layers.Concatenate()([title, text_body, tags])
#apply a intermidiate layer to recombine input features into richer representation
features = layers.Dense(64, activation="relu")(features)

#define model outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
	num_departments, activation="softmax", name="department")(features)

model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

### TRAINING A MULTI-INPUT, MULTI-OUTPUT MODEL

In [3]:
import numpy as np

num_samples = 1280

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(
	optimizer="rmsprop",
	loss=["mean_squared_error", "categorical_crossentropy"],
	metrics=[["mean_absolute_error"], ["accuracy"]]
)

model.fit(
	[title_data, text_body_data, tags_data],
	[priority_data, department_data],
	epochs=3
)

model.evaluate([title_data, text_body_data, tags_data], [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

Epoch 1/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - department_accuracy: 0.2602 - department_loss: 32.0590 - loss: 32.3731 - priority_loss: 0.3141 - priority_mean_absolute_error: 0.4823
Epoch 2/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - department_accuracy: 0.2484 - department_loss: 32.2761 - loss: 32.6077 - priority_loss: 0.3316 - priority_mean_absolute_error: 0.5001
Epoch 3/3
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - department_accuracy: 0.2547 - department_loss: 39.4007 - loss: 39.7324 - priority_loss: 0.3316 - priority_mean_absolute_error: 0.5001
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - department_accuracy: 0.1391 - department_loss: 27.1702 - loss: 27.5019 - priority_loss: 0.3316 - priority_mean_absolute_error: 0.5001
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


You can plot a Functional model as a graph with the plot_model()

In [24]:
import keras
import pydot
import graphviz
import tensorflow as tf

keras.utils.plot_model( model, "ticket_classifier_with_shape_info.png", show_shapes=True)

You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [ ]:
model.layers

[<InputLayer name=title, built=True>,
 <InputLayer name=text_body, built=True>,
 <InputLayer name=tags, built=True>,
 <Concatenate name=concatenate, built=True>,
 <Dense name=dense_2, built=True>,
 <Dense name=priority, built=True>,
 <Dense name=department, built=True>]

In [26]:
model.layers[3].input

[<KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=title>,
 <KerasTensor shape=(None, 10000), dtype=float32, sparse=False, ragged=False, name=text_body>,
 <KerasTensor shape=(None, 100), dtype=float32, sparse=False, ragged=False, name=tags>]

In [27]:
model.layers[3].output

<KerasTensor shape=(None, 20100), dtype=float32, sparse=False, ragged=False, name=keras_tensor_2>

feature extraction, creating models that reuse intermediate features from another model.

Let’s say you want to add another output to the previous model—you want to estimate how long a given issue ticket will take to resolve, a kind of difficulty rating. You could do this via a classification layer over three categories: “quick,” “medium,” and “difficult.” You don’t need to recreate and retrain a model from scratch. You can start from the intermediate features of your previous model, since you have access to them, like this.

In [ ]:
features = model.layers[4].output #layers[4] is the output of the previous model
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model=keras.Model(
	inputs=[title, text_body, tags],
	outputs=[priority, department, difficulty]
)


AttributeError: module 'keras' has no attribute 'model'